# Optimización de parámetros


En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

In [3]:
df.head()

,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,barrio_match,PH,apartment,house,...,VILLA LUGANO,VILLA LURO,VILLA ORTUZAR,VILLA PUEYRREDON,VILLA REAL,VILLA RIACHUELO,VILLA SANTA RITA,VILLA SOLDATI,VILLA URQUIZA,outlier_price_m2
0,-34.589,-58.417,170000.000,40.000,38.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-34.591,-58.418,90000.000,27.000,27.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-34.587,-58.437,150000.000,44.000,44.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-34.593,-58.428,154000.000,58.000,58.000,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-34.593,-58.428,154000.000,58.000,58.000,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.shape

(6376, 59)

In [5]:
# Hacé la separación en esta celda

In [6]:
# Esto es lo que hace
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

In [7]:
# Creá en esta celda la variable param_grid
param_grid = [{'max_depth':[1,2,3,4,5],'max_features':[1,2,3,4,5]},]

In [8]:
param_grid

[{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}]

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_reg,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score='boolean')

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [10]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5],
                          'max_features': [1, 2, 3, 4, 5]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score='boolean',
             sc

In [11]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

In [12]:
# Mostrá los grid_scores en esta celda
grid_search.cv_results_

{'mean_fit_time': array([0.00559678, 0.00459571, 0.01238751, 0.00719304, 0.00699301,
        0.00559459, 0.00459552, 0.00579424, 0.00679336, 0.00479498,
        0.00619349, 0.00479522, 0.00439601, 0.00639358, 0.00420284,
        0.00438929, 0.00499325, 0.00439281, 0.00459566, 0.00620074,
        0.00499501, 0.00499511, 0.00539446, 0.00639362, 0.00659328]),
 'std_fit_time': array([0.00120121, 0.00101911, 0.00515596, 0.00116549, 0.00227803,
        0.00185297, 0.00048949, 0.00171858, 0.00263514, 0.00159829,
        0.00132513, 0.00074786, 0.00101883, 0.00233029, 0.00039609,
        0.00048266, 0.00063498, 0.0004914 , 0.00048951, 0.00097733,
        0.00063151, 0.00063181, 0.00101848, 0.00101864, 0.00162318]),
 'mean_score_time': array([0.00119877, 0.00119867, 0.00219769, 0.0017982 , 0.00199809,
        0.00179801, 0.00119848, 0.00159836, 0.00119886, 0.00219808,
        0.00199757, 0.00119853, 0.00099874, 0.00099874, 0.00079579,
        0.00160484, 0.00099878, 0.00119777, 0.0009984 , 0.00

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

In [13]:
grid_search.best_score_

-713698124.9712428

In [14]:
grid_search.best_params_

{'max_depth': 4, 'max_features': 4}

Convertimos a RMSE.

In [41]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [42]:
grid_search.best_score_

-459407773.74392384

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [29]:
param_grid = [
    {"min_samples_split": [2, 10, 20],'max_depth': [None, 2, 5, 10, 15],"min_samples_leaf": [1, 5, 10, 15],"max_leaf_nodes": [None, 5, 10, 20]},]

In [30]:
tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_reg,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score='boolean')

In [31]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid=[{'max_depth': [None, 2, 5, 10, 15],
                          'max_leaf_nodes': [None, 5, 10, 20],
                          'min_samples_leaf': [1, 5, 10, 15],
                          

In [43]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

In [44]:
results = pd.DataFrame(grid_search.cv_results_, columns=['rank_test_score','params','mean_test_score','std_test_score','mean_train_score','std_train_score'])
results.sort_values(by=['rank_test_score'],inplace=True)
results.head()

,rank_test_score,params,mean_test_score,std_test_score,mean_train_score,std_train_score
155,1,"{'max_depth': 10, 'max_leaf_nodes': None, 'min...",-459407773.744,21980190.436,-333393912.892,3185225.003
154,1,"{'max_depth': 10, 'max_leaf_nodes': None, 'min...",-459407773.744,21980190.436,-333393912.892,3185225.003
153,1,"{'max_depth': 10, 'max_leaf_nodes': None, 'min...",-459407773.744,21980190.436,-333393912.892,3185225.003
203,4,"{'max_depth': 15, 'max_leaf_nodes': None, 'min...",-465239871.772,23937456.805,-319688176.708,3000271.997
201,4,"{'max_depth': 15, 'max_leaf_nodes': None, 'min...",-465239871.772,23937456.805,-319688176.708,3000271.997


In [33]:
grid_search.cv_results_

{'mean_fit_time': array([0.03136396, 0.02157393, 0.0203764 , 0.0201818 , 0.01937633,
        0.0183845 , 0.0175808 , 0.01698427, 0.01718473, 0.01598501,
        0.01597786, 0.01678209, 0.00959253, 0.00938778, 0.0083961 ,
        0.00899096, 0.00878901, 0.00859418, 0.00898643, 0.00839019,
        0.00918508, 0.00899358, 0.00899024, 0.00899038, 0.01058884,
        0.01058927, 0.01098495, 0.0109901 , 0.01058612, 0.01099105,
        0.01118736, 0.01038771, 0.0107861 , 0.01079035, 0.01038623,
        0.01118746, 0.01238856, 0.01258116, 0.01258745, 0.01218352,
        0.01298742, 0.01258602, 0.01218634, 0.01258707, 0.01218791,
        0.01217794, 0.01218066, 0.01218619, 0.00699801, 0.00679488,
        0.00719123, 0.00699801, 0.00679116, 0.00699239, 0.00679283,
        0.00699377, 0.00679412, 0.00679221, 0.00699115, 0.00719409,
        0.00699277, 0.00679431, 0.00658746, 0.00639415, 0.00678735,
        0.00719461, 0.00659094, 0.0067894 , 0.00679231, 0.0063931 ,
        0.00679545, 0.00678554,

In [34]:
grid_search.best_params_

{'max_depth': 10,
 'max_leaf_nodes': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2}

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [35]:
optimised_decision_tree = grid_search.best_estimator_

In [36]:
optimised_decision_tree

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=15,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [37]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [38]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [39]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [40]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
